In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# مرحباً بالعالم: دائرتك الكمية الأولى

أنشئ [حالة بِل](https://en.wikipedia.org/wiki/Bell_state) (qubitان متشابكان) وشغِّلها بثلاث طرق:

1. **المحاكاة المثالية** — نتائج مثالية، لا حاجة إلى حساب
2. **المحاكاة الضوضائية** — تحاكي جهازاً حقيقياً، لا حاجة إلى حساب
3. **العتاد الكمي الحقيقي** — يتطلب [حساب IBM Quantum](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account)

## بناء الدائرة

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw(output="mpl")

## الخيار الأول: المحاكاة المثالية (لا حاجة إلى حساب)
يستخدم `StatevectorSampler` — محاكياً محلياً يُعطي نتائج مثالية خالية من الضوضاء.

In [ ]:
from qiskit.primitives import StatevectorSampler

result = StatevectorSampler().run([qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

## الخيار الثاني: المحاكاة الضوضائية (لا حاجة إلى حساب)
يستخدم `FakeManilaV2` — محاكياً محلياً يُحاكي جهاز IBM الكمي الحقيقي، بما في ذلك خصائص الضوضاء الخاصة به. يجب أولاً تحويل الدائرة (transpile) لتتوافق مع مجموعة البوابات واتصالية qubit الخاصة بالجهاز.

In [ ]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeManilaV2()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## الخيار الثالث: العتاد الكمي الحقيقي
يتطلب حساب IBM Quantum. راجع [إعداد حساب IBM Quantum](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account) لمزيد من التفاصيل.

إذا لم تكن قد حفظت بيانات الاعتماد بعد في جلسة Binder هذه، فشغّل هذا أولاً:

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token="<your-api-key>",
    instance="<your-crn>",
    overwrite=True
)

**ملاحظة:** قد تستغرق المهام على العتاد الحقيقي بعض الوقت بحسب أوقات الانتظار في قائمة الانتظار. إذا كانت الخلية لا تزال تعمل، يمكنك التحقق من حالة مهمتك وعرض النتائج على [quantum.cloud.ibm.com/workloads](https://quantum.cloud.ibm.com/workloads?user=me).

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Running on {backend.name}")

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## ما الخطوة التالية؟
- **[الدروس التعليمية](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=docs/tutorials)** — أدلة خطوة بخطوة حول الخوارزميات، وتخفيف الأخطاء، والتحويل (transpilation)، وغير ذلك
- **[المسارات الدراسية](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=learning/courses)** — مسارات تعلُّم منظَّمة من أساسيات الكم إلى الحوسبة على نطاق واسع
- **[وضع الاختبار المحلي](https://janlahmann.github.io/Qiskit-documentation/#no-token-use-local-testing-mode)** — تشغيل معظم دفاتر الملاحظات دون حساب IBM Quantum